In [65]:
import airfoils as af
import matplotlib.pyplot as plt
import numpy as np
import potential as pot
from wing import Wing

# Airfoil

In [66]:
airfoil = af.Airfoil.NACA4("2412", n_points=200)
af_xs, af_ys = airfoil.all_points
plt.scatter(af_xs, af_ys, 1)
plt.axis('scaled')

alpha0 = -2 * np.pi / 180

# Wing Geometry

In [67]:
def linSpan(sp, Ni):
    return np.linspace(0, sp, Ni)

def uniformChord(Ni, ch=1):
    return ch * np.ones(Ni)

In [68]:
N = 51
M = 3
%matplotlib qt
wg = Wing(
    airfoil=airfoil,
    isSymmetric=False,
    span=6/2,
    alpha=2,
    beta=0,
    Ni=N,
    Nj=M,
    sweepOffset=0,
    dihAngle=0,
    chordFun=uniformChord,
    spanFun=linSpan
)

## Wind

In [69]:
Umag = 16
dens = 1.2

# Symmetric Wing

In [70]:
def symm_wing(x, y, z, i, j, panel):

    U1, U1st = pot.hshoeSL2(x, y, z, i, j, panel)
    U2 , U2st = pot.hshoeSL2(x, -y, z, i, j, panel)

    U_ind = np.array([U1[0] + U2[0],
                      U1[1] - U2[1],
                      U1[2] + U2[2]])
    U_ind_st = np.array([U1st[0] + U2st[0],
                         U1st[1] - U2st[1],
                         U1st[2] + U2st[2]])
    return U_ind , U_ind_st

In [71]:
L_sym, D_sym, Gammas, w_ind = wg.solveWing(alpha0, Umag, dens, symm_wing)
print(f"For the whole Wing\nL:{2*L_sym}\t|\tD:{2*D_sym}")
print(f"CL:{L_sym/(dens*(Umag**2)*wg.wingArea)}\t|\tCD_ind:{D_sym/(dens*(Umag**2)*wg.wingArea)}")


For the whole Wing
L:293.6589808788896	|	D:5.119690380145868
CL:0.15932019361918925	|	CD_ind:0.002777609798256222


In [72]:
LvsSweep = []
DvsSweep = []
GammavsSweep = []
WindvsSweep = []
offsets = np.linspace(0, 3,10)
for offset in offsets:
    wg.set_offset(offset)
    L , D , Gamma , w = wg.solveWing(alpha0,Umag,dens,symm_wing)
    LvsSweep.append(L)
    DvsSweep.append(D)
    GammavsSweep.append(Gamma)
    WindvsSweep.append(w)
    
LvsSweep = np.array(LvsSweep)
DvsSweep = np.array(DvsSweep)
GammavsSweep = np.array(GammavsSweep)
WindvsSweep = np.array(WindvsSweep)


In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(14, 6))
fig.suptitle('Sweep effect on Lift And Drag')

ax1.plot(offsets, 2* LvsSweep)
ax1.set_xlabel("Sweep Offset")
ax1.set_title("Lift")
ax1.minorticks_on()
ax1.grid()

ax2.plot(offsets, 2*DvsSweep)
ax2.set_xlabel("Sweep Offset")
ax2.set_title("Drag")
ax2.minorticks_on()
ax2.grid()

ax3.plot(offsets, LvsSweep / DvsSweep)
ax3.set_xlabel("Sweep Offset")
ax3.set_title("L/D")
ax3.minorticks_on()
ax3.grid()

fig.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
fig.suptitle('Results')
for i, offset in enumerate(offsets):
    ax1.plot((wg.Dspan[1:] + wg.Dspan[:-1])/2, GammavsSweep[i], label="offset {:.2f}".format(offset))
    ax1.plot(-(wg.Dspan[1:] + wg.Dspan[:-1])/2, GammavsSweep[i])

    ax1.set_xlabel("span")
    ax1.set_title("Gamma")

    # (w_ind/Umag) * 180 / np.pi)
    ax2.plot((wg.Dspan[1:] + wg.Dspan[:-1])/2, WindvsSweep[i], label="offset {:.2f}".format(offset))
    ax2.plot(-(wg.Dspan[1:] + wg.Dspan[:-1])/2, WindvsSweep[i] )

    ax2.set_xlabel("span")
    ax2.set_title("w_ind")
ax1.legend(loc="lower right")
ax1.grid(which='minor', linewidth=0.1)
ax1.grid(which='major', linewidth=1)
ax1.minorticks_on()

ax2.legend(loc="upper right")
ax2.grid(which='minor', linewidth = 0.1)
ax2.grid(which='major', linewidth = 1)
ax2.minorticks_on()
fig.show()

In [ ]:
LvsAOA = []
DvsAOA = []
GammavsAOA = []
WindvsAOA = []
angles = np.arange(-5, 15)
wg.set_offset(0)

for angle in angles:
    wg.set_alpha(angle)
    L, D, Gamma, w = wg.solveWing(alpha0, Umag, dens, symm_wing)
    LvsAOA.append(L)
    DvsAOA.append(D)
    GammavsAOA.append(Gamma)
    WindvsAOA.append(w)
    
LvsAOA = np.array(LvsAOA)
DvsAOA = np.array(DvsAOA)
GammavsAOA = np.array(GammavsAOA)
WindvsAOA = np.array(WindvsAOA)


In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
fig.suptitle('Angle of Attack Effect on Lift And Drag')

ax1.plot(angles, 2*LvsAOA)
ax1.set_xlabel("AoA")
ax1.set_title("Lift")
ax1.minorticks_on()
ax1.grid()

ax2.plot(angles, 2*DvsAOA)
ax2.set_xlabel("AoA")
ax2.set_title("Drag")
ax2.minorticks_on()
ax2.grid()
fig.show()